In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns',500)


In [2]:
df = pd.read_csv('data/pbp_data_mvp.csv')
df['Date'] = pd.to_datetime(df["Date"])

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14,16,86,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
team_list = df.home_team.unique()
year_list = df.Date.dt.year.unique()

These are rows that do not have any impact on the game in a way that can be useful for prediction. 

In [4]:
cols_to_drop = ['Unnamed: 0','forfeit_info','lf_ump_id','rf_ump_id','protest_info',
                    'date_game_completed','additional_info','save_pitch_id',
                    'game_win_rbi_batter_id','game_in_series','away_catch_interference',
                'home_catch_interference','away_pitch_balks','home_pitch_balks']
drop_cols_before_modeling = ['day_of_week','away_league',
                            'away_team_game_number','home_league',
                            'home_team_game_number','day_or_night','park_id','attendance',
                            'time_of_game','away_line_scores','home_line_scores','year','id',
                            'outcome']

df.drop(columns=cols_to_drop,inplace=True)
df.drop(columns=drop_cols_before_modeling,inplace=True)
df.drop(df.loc[:,'hb_ump_id':'acquisition_info'],axis=1,inplace=True)

Now I must get rid of all statistics that are not able to be averaged to create cumulative statistics over a year.

In [5]:
df.head(1)

,Date,away_team,home_team,away_team_score,home_team_score,number_of_outs,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_OBP,away_OBP,home_AVG,away_AVG,home_singles,away_singles,home_SLG,away_SLG,home_BABIP,away_BABIP,home_ISO,away_ISO,home_PASO,away_PASO,home_total_bases,away_total_bases,home_runs_created,away_runs_created,home_wOBA,away_wOBA,game_number_of_season,home_won_last_game,away_won_last_game,target,away_outcome,home_outcome
0,2000-03-30,NYN,CHN,5,1,66,37,6,2,0,1,5,1,1,1,8,0,5,1,0,0,10,5,0,0,0,33,14,2,0,2,0,36,5,0,0,0,0,2,0,0,6,1,9,0,0,2,10,7,5,5,0,33,14,0,0,0,0,0.261905,0.319149,0.138889,0.162162,5,3,0.138889,0.297297,0.185185,0.15625,0.0,0.135135,4.0,7.4,5,11,1.309524,3.422222,0.209512,0.288298,1,0,0,0,1,0


In [6]:
drop_for_cummean = ['number_of_outs','target','game_number_of_season',
                   'away_won_last_game','home_won_last_game','game_number_of_season']
df.drop(columns=drop_for_cummean,inplace=True)

In [7]:
# making dummy rows for use in putting games into year/team buckets
df['home_date'] = df.Date
df['away_date'] = df.Date

In [8]:
# initialize dict so that we can aggregate the stats of each team per year
# final dict is where we are going to be adding the aggregated stats
stat_dict = {}
final_dict = {}
for year in year_list:
    stat_dict[year] = {}
    final_dict[year] = {}
    for team in team_list:
        stat_dict[year][team] = []
        final_dict[year][team] = []

In [9]:
# this is putting games into the correct year/team combo
# we need both home and away games bc away stats obviously affect stats
# of the team
# this will end in lists that contain every game for every team for every year
for year in year_list:
    for team in team_list:
        # home game stats
        for game in df[(df.Date.dt.year == year)&(df.home_team == team)].filter(regex='home').values.tolist():
            stat_dict[year][team].append(np.array(game))
        # away game stats
        for game in df[(df.Date.dt.year == year)&(df.away_team == team)].filter(regex='away').values.tolist():
            stat_dict[year][team].append(np.array(game))
            

In [10]:
from operator import itemgetter

In [11]:
# have to sort each year/team array so that they are in the correct date order
for year in year_list:
    for team in team_list:
        stat_dict[year][team] = sorted(stat_dict[year][team],key=itemgetter(-1))
        #print((stat_dict[year][team]))

In [12]:
# putting date in the front to make it easier to work with
for year in year_list:
    for team in team_list:
        year_team_stats = []
        for game in stat_dict[year][team]:
            #print(game[-1])
            #print(game[:-1])
            #print(np.insert(game[:-1],0,game[-1]))
            year_team_stats.append(np.insert(game[:-1],0,game[-1]))
        stat_dict[year][team] = year_team_stats

In [13]:
# this is aggregating the stats per year
# so each game is the mean of all stats of that game and all previous
# I am going to need to 
for year in year_list:
    for team in team_list:
        curr_game_number = 1
        aggregate_stats = np.array(stat_dict[year][team][0][2:])
        for game in stat_dict[year][team]:
            header_info = game[0:2]
            contents = game[2:]
            aggregate_stats_current_game = aggregate_stats/curr_game_number
            aggregate_stats = contents+aggregate_stats
            final_dict[year][team].append(np.concatenate((header_info,aggregate_stats_current_game)))
            curr_game_number +=1

In [14]:
agg_df = pd.DataFrame()

In [15]:
# creating the aggregate stats array
agg_array = []
for year in year_list:
    for team in team_list:
        for game in final_dict[year][team]:
            agg_array.append(game)

In [16]:
# making the array of all stats
agg_df = pd.DataFrame(agg_array)

In [17]:
away_data_column_names = list(df.filter(regex='away').columns)[1:-1] # gets rid of team name and date
home_data_column_names = list(df.filter(regex='home').columns)[1:-1] # gets rid of team name and date

In [18]:
game_basic_info_df = df.iloc[:,0:3]
for stat in away_data_column_names:
    game_basic_info_df[stat] = ''
for stat in home_data_column_names:
    game_basic_info_df[stat] = ''

In [19]:
final_matrix_for_df = []

In [20]:
for index,row in game_basic_info_df.iterrows():
    # append home team aggregate stats
    home_data = agg_df[(agg_df[0] == row.Date)&(agg_df[1]==row.home_team)].iloc[:,2:].to_numpy()
    # append away team aggregate stats
    away_data = agg_df[(agg_df[0] == row.Date)&(agg_df[1]==row.away_team)].iloc[:,2:].to_numpy()
    # create the full row
    full_row = np.concatenate((home_data[0],away_data[0]))
    temp_array = [row.Date,row.home_team,row.away_team]
    for individual_stat in full_row:
        temp_array.append(individual_stat)
    final_matrix_for_df.append(temp_array)
    #game_basic_info_df[(game_basic_info_df.home_team == row.home_team)&(game_basic_info_df.Date == row.Date)].iloc[:,3:] = full_row
    #row.iloc[3:] = full_row

In [21]:
game_basic_info_df[(game_basic_info_df.home_team == row.home_team)&(game_basic_info_df.Date == row.Date)].iloc[:,3:]

,away_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,away_OBP,away_AVG,away_singles,away_SLG,away_BABIP,away_ISO,away_PASO,away_total_bases,away_runs_created,away_wOBA,away_outcome,home_team_score,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_OBP,home_AVG,home_singles,home_SLG,home_BABIP,home_ISO,home_PASO,home_total_bases,home_runs_created,home_wOBA,home_outcome
48567,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [22]:
final_final_df = pd.DataFrame(final_matrix_for_df)

In [23]:
df[((df['home_team']=='ATL')|(df['away_team']=='ATL'))&(df['Date'].dt.year == 2000)].head(5)

,Date,away_team,home_team,away_team_score,home_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_OBP,away_OBP,home_AVG,away_AVG,home_singles,away_singles,home_SLG,away_SLG,home_BABIP,away_BABIP,home_ISO,away_ISO,home_PASO,away_PASO,home_total_bases,away_total_bases,home_runs_created,away_runs_created,home_wOBA,away_wOBA,away_outcome,home_outcome,home_date,away_date
2,2000-04-03,COL,ATL,0,2,31,6,2,0,0,0,1,0,0,2,2,7,0,0,1,7,3,2,2,1,24,10,0,0,1,0,30,7,0,0,2,2,0,0,1,1,0,6,1,0,1,6,2,0,0,0,27,12,0,0,1,0,0.281250,0.242424,0.233333,0.193548,5,4,0.433333,0.258065,0.227273,0.250000,0.200000,0.064516,5.000000,4.428571,13,8,3.354839,1.939394,0.314375,0.241290,0,1,2000-04-03,2000-04-03
18,2000-04-04,COL,ATL,5,3,36,11,4,1,0,5,1,0,0,5,2,6,2,1,0,10,3,3,3,1,27,18,2,0,0,0,33,6,2,0,1,3,0,0,1,4,2,4,1,1,0,8,5,5,5,0,27,10,0,0,0,0,0.289474,0.390244,0.181818,0.305556,3,6,0.333333,0.472222,0.178571,0.366667,0.151515,0.166667,8.250000,6.000000,11,17,2.972973,6.634146,0.299722,0.397179,1,0,2000-04-04,2000-04-04
27,2000-04-05,COL,ATL,6,9,34,9,0,1,2,4,3,0,0,1,0,4,0,0,1,5,4,6,6,0,24,8,2,0,1,0,36,12,1,0,1,6,0,0,2,5,0,4,0,0,1,10,4,5,5,0,27,12,2,0,1,0,0.441860,0.285714,0.333333,0.264706,10,6,0.444444,0.500000,0.354839,0.250000,0.111111,0.235294,9.000000,8.500000,16,17,6.634146,4.857143,0.399070,0.338571,0,1,2000-04-05,2000-04-05
54,2000-04-07,SFN,ATL,6,2,34,8,1,0,3,5,0,1,0,2,0,5,0,0,1,4,2,2,2,1,27,4,0,0,1,0,36,10,1,0,2,2,1,0,0,2,0,6,0,0,1,10,5,5,5,0,27,10,0,1,1,0,0.315789,0.270270,0.277778,0.235294,7,4,0.472222,0.529412,0.285714,0.185185,0.194444,0.294118,6.000000,6.800000,17,18,5.368421,5.000000,0.344211,0.338108,1,0,2000-04-07,2000-04-07
69,2000-04-08,SFN,ATL,5,7,34,10,0,0,1,5,0,0,1,1,0,6,1,0,3,4,5,7,7,0,24,7,0,0,0,0,32,11,1,1,1,7,2,0,1,4,0,6,1,0,0,8,3,5,5,1,27,13,0,0,3,0,0.432432,0.333333,0.343750,0.294118,8,9,0.531250,0.382353,0.400000,0.333333,0.187500,0.088235,5.333333,5.666667,17,13,7.083333,4.085714,0.421351,0.320000,0,1,2000-04-08,2000-04-08


In [24]:
# so this isnt actually cumulative its just total so far + this row / 2 which isnt how math works
final_final_df[((final_final_df['home_team']=='ATL')|(final_final_df['away_team']=='ATL'))&(final_final_df['Date'].dt.year == 2000)].head(5)

KeyError: 'home_team'

In [30]:
final_final_df.to_csv('data/aggregate_data.csv')

In [26]:
columns_for_output_df = ['Date','home_team','away_team']

In [27]:
for col_name in home_data_column_names:
    columns_for_output_df.append(col_name)
for col_name in away_data_column_names:
    columns_for_output_df.append(col_name)


In [28]:
final_final_df.columns = columns_for_output_df

In [29]:
final_final_df

,Date,home_team,away_team,home_team_score,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_OBP,home_AVG,home_singles,home_SLG,home_BABIP,home_ISO,home_PASO,home_total_bases,home_runs_created,home_wOBA,home_outcome,away_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,away_OBP,away_AVG,away_singles,away_SLG,away_BABIP,away_ISO,away_PASO,away_total_bases,away_runs_created,away_wOBA,away_outcome
0,2000-03-30,CHN,NYN,1.000000,36.000000,5.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,6.000000,1.000000,9.000000,0.000000,0.000000,2.000000,10.000000,7.000000,5.000000,5.000000,0.000000,33.000000,14.000000,0.000000,0.000000,0.000000,0.0,0.261905,0.138889,5.000000,0.138889,0.185185,0.000000,4.000000,5.000000,1.309524,0.209512,0.000000,5.000000,37.000000,6.000000,2.000000,0.000000,1.000000,5.000000,1.000000,1.000000,1.000000,8.000000,0.000000,5.000000,1.000000,0.000000,0.000000,10.000000,5.000000,0.000000,0.000000,0.000000,33.000000,14.000000,2.000000,0.000000,2.000000,0.000000,0.319149,0.162162,3.000000,0.297297,0.156250,0.135135,7.400000,11.000000,3.422222,0.288298,1.000000
1,2000-04-03,SLN,CHN,7.000000,34.000000,10.000000,1.000000,1.000000,3.000000,7.000000,0.000000,0.000000,0.000000,7.000000,0.000000,6.000000,3.000000,0.000000,0.000000,10.000000,4.000000,1.000000,1.000000,0.000000,27.000000,8.000000,0.000000,0.000000,1.000000,0.0,0.414634,0.294118,5.000000,0.647059,0.280000,0.352941,5.666667,22.000000,9.121951,0.450488,1.000000,1.000000,36.000000,5.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,6.000000,1.000000,9.000000,0.000000,0.000000,2.000000,10.000000,7.000000,5.000000,5.000000,0.000000,33.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.261905,0.138889,5.000000,0.138889,0.185185,0.000000,4.000000,5.000000,1.309524,0.209512,0.000000
2,2000-04-03,ATL,COL,2.000000,30.000000,7.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,6.000000,1.000000,0.000000,1.000000,6.000000,2.000000,0.000000,0.000000,0.000000,27.000000,12.000000,0.000000,0.000000,1.000000,0.0,0.281250,0.233333,5.000000,0.433333,0.227273,0.200000,5.000000,13.000000,3.354839,0.314375,1.000000,0.000000,31.000000,6.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,7.000000,0.000000,0.000000,1.000000,7.000000,3.000000,2.000000,2.000000,1.000000,24.000000,10.000000,0.000000,0.000000,1.000000,0.000000,0.242424,0.193548,4.000000,0.258065,0.250000,0.064516,4.428571,8.000000,1.939394,0.241290,0.000000
3,2000-04-03,CIN,MIL,3.000000,19.000000,5.000000,1.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,0.000000,16.000000,8.000000,2.000000,0.000000,0.000000,0.0,0.300000,0.263158,3.000000,0.473684,0.285714,0.210526,4.750000,9.000000,2.700000,0.336500,-1.000000,3.000000,22.000000,7.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,5.000000,0.000000,1.000000,1.000000,0.000000,0.000000,8.000000,1.000000,3.000000,3.000000,0.000000,15.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.444444,0.318182,6.000000,0.363636,0.333333,0.045455,22.000000,8.000000,3.555556,0.372593,-1.000000
4,2000-04-03,MIA,SFN,6.000000,36.000000,12.000000,3.000000,0.000000,0.000000,5.000000,0.000000,0.000000,1.0000